In [ ]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --cluster-id j-34LUZUJYIUCH5 --auth-type None --language python  

In [ ]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

In [ ]:
sc.list_packages()

In [ ]:
#sc.install_pypi_package("xgboost==1.7.2")
#sc.install_pypi_package("scikit-learn")
sc.install_pypi_package("plotly")
#sc.install_pypi_package("seaborn")

In [ ]:
%%info

In [ ]:
spark

In [ ]:
# df_ride_fare = spark.read.option("recursiveFileLookup", "true").parquet("s3://dsoaws/nyc-taxi-orig-cleaned-split-parquet-all-years/ride-fare/")
df_ride_fare = spark.read.option("recursiveFileLookup", "true").parquet("s3://dsoaws/nyc-taxi-orig-cleaned-split-parquet-per-year-multiple-files/ride-fare/year=2019")
df_ride_fare

In [ ]:
# df_ride_info = spark.read.option("recursiveFileLookup", "true").parquet("s3://dsoaws/nyc-taxi-orig-cleaned-split-parquet-all-years/ride-info/")
df_ride_info = spark.read.option("recursiveFileLookup", "true").parquet("s3://dsoaws/nyc-taxi-orig-cleaned-split-parquet-per-year-multiple-files/ride-info/year=2019")
df_ride_info

In [ ]:
df_joined = df_ride_fare.join(df_ride_info, on="ride_id")
df_joined

In [ ]:
df_joined.printSchema()

In [ ]:
df_dropped = df_joined.drop("ride_id").drop("pickup_at").drop("dropoff_at").drop("rate_code_id").drop("store_and_fwd_flag")
df_dropped

In [ ]:
df_dropped.printSchema()

In [ ]:
df_selected = df_dropped.select('passenger_count', 'total_amount').limit(10000000) #.groupby('passenger_count')
df_selected

In [ ]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt


plt.clf()

#df_grouped = df_dropped.limit(100000000).select('passenger_count', 'total_amount').toPandas().groupby('passenger_count').total_amount.mean()
df_selected.toPandas().groupby('passenger_count').total_amount.mean().plot(kind='bar')

plt.xlabel('Passenger Count')
plt.ylabel('Avg Total Amount')

plt.title('Passenger Count vs. Avg Total Amount')
plt.show()

%matplot plt

In [ ]:
# Move 'total_amount' to the first column
df_columns = df_dropped.columns
feature_columns = [feature for feature in df_columns if feature != 'total_amount']
df = df_dropped.select(['total_amount'] + feature_columns)

# Remove all rows with negative `total_amount`
df = df[(df.total_amount >= 0)]
df.show(5)

In [ ]:
# Split the dataset randomly into 70% for training and 30% for testing. Passing a seed for deterministic behavior
df_train, df_validation = df.randomSplit([0.7, 0.3], seed = 0)
print("There are %d training examples and %d test examples." % (df_train.count(), df_validation.count()))

In [ ]:
%%local 

import sagemaker

sess = sagemaker.Session()
output_bucket = sess.default_bucket()

prefix = f'nyc-taxi-2019-sagemaker-emr'
output_key_prefix = f"{prefix}"
print(f'Output S3: s3://{output_bucket}/{output_key_prefix}')

train_output_s3_uri = f's3://{output_bucket}/{output_key_prefix}/train'
validation_output_s3_uri = f's3://{output_bucket}/{output_key_prefix}/validation'
print(f'Output S3 for train set: {train_output_s3_uri}')
print(f'Output S3 for validation set: {validation_output_s3_uri}')


Send the following variables (s3 paths) to spark

In [ ]:
%%send_to_spark -i train_output_s3_uri -t str -n train_output_s3_uri

In [ ]:
%%send_to_spark -i validation_output_s3_uri -t str -n validation_output_s3_uri

In [ ]:
# Save the dataset to S3
df_train.write.option("mode", "overwrite").parquet(train_output_s3_uri)
df_validation.write.option("mode", "overwrite").parquet(validation_output_s3_uri)

print(f'Output train data s3 URI: {train_output_s3_uri}')
print(f'Output validation data s3 URI: {validation_output_s3_uri}')